**Todo**

-   doublons ×
-   valeurs vides (whitespace only) ×
-   caractères spéciaux ×
-   url ×
-   espaces multiples ×
-   balises html ×
-   unescape (gérer les \&amp; et autres) ×
-   dates en datetime ×
-   lower ×


In [33]:
import pandas as pd
import re
from html import unescape

## Chargement


In [ ]:
fake_df = pd.read_csv('../data/fake.csv')
true_df = pd.read_csv('../data/True.csv')
fake_df['label'] = 'fake'
true_df['label'] = 'true'
true_df.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",true
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",true
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",true
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",true
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",true


# Exploration


In [ ]:
fake_df.describe()

,title,text,subject,date,label
count,23481,23481,23481,23481,23481
unique,17903,17455,6,1681,1
top,MEDIA IGNORES Time That Bill Clinton FIRED His...,,News,"May 10, 2017",fake
freq,6,626,9050,46,23481


In [ ]:
true_df.describe()

In [ ]:
fake_df.info()

In [ ]:
print(fake_df['subject'].value_counts())
print('')
print(true_df['subject'].value_counts())

In [ ]:
fake_df[fake_df['text'].duplicated()]

In [ ]:
fake_df[fake_df['title'].duplicated()]

In [ ]:
display(fake_df[fake_df['text'].str.isspace()])
display(true_df[true_df['text'].str.isspace()])

In [ ]:
display(fake_df[fake_df['title'].str.isspace()])
display(true_df[true_df['title'].str.isspace()])

In [ ]:
display(fake_df[fake_df['date'].str.isspace()])
display(true_df[true_df['date'].str.isspace()])

In [ ]:
fake_df.isna().sum()

In [ ]:
true_df.isna().sum()

# Nettoyage


## Supprimer les doublons


In [ ]:
clean_fake_df = fake_df.copy()
clean_true_df = true_df.copy()

In [ ]:
clean_fake_df = clean_fake_df.drop_duplicates('text')
clean_fake_df.describe()

In [ ]:
clean_true_df = clean_true_df.drop_duplicates('text')
clean_true_df.describe()

In [ ]:
def remove_if_brackets(text):
    if text.strip().startswith('[') and text.strip().endswith(']'):
        return ''
    return text


clean_fake_df['text'] = clean_fake_df['text'].apply(lambda x: remove_if_brackets(x))
clean_fake_df['title'] = clean_fake_df['title'].apply(lambda x: remove_if_brackets(x))

clean_true_df['text'] = clean_true_df['text'].apply(lambda x: remove_if_brackets(x))
clean_true_df['title'] = clean_true_df['title'].apply(lambda x: remove_if_brackets(x))

In [ ]:
def normalize_spaces(text):
    return re.sub(r'\s+', ' ', text).strip()


clean_fake_df['text'] = clean_fake_df['text'].apply(lambda x: normalize_spaces(x))
clean_fake_df['title'] = clean_fake_df['title'].apply(lambda x: normalize_spaces(x))

clean_true_df['text'] = clean_true_df['text'].apply(lambda x: normalize_spaces(x))
clean_true_df['title'] = clean_true_df['title'].apply(lambda x: normalize_spaces(x))

In [ ]:
def remove_special_characters(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)


clean_fake_df['text'] = clean_fake_df['text'].apply(
    lambda x: remove_special_characters(x)
)
clean_fake_df['title'] = clean_fake_df['title'].apply(
    lambda x: remove_special_characters(x)
)

clean_true_df['text'] = clean_true_df['text'].apply(
    lambda x: remove_special_characters(x)
)
clean_true_df['title'] = clean_true_df['title'].apply(
    lambda x: remove_special_characters(x)
)

In [ ]:
def remove_urls(text):
    return re.sub(r'http\S+|www\S+', '', text)


clean_fake_df['text'] = clean_fake_df['text'].apply(lambda x: remove_urls(x))
clean_fake_df['title'] = clean_fake_df['title'].apply(lambda x: remove_urls(x))

clean_true_df['text'] = clean_true_df['text'].apply(lambda x: remove_urls(x))
clean_true_df['title'] = clean_true_df['title'].apply(lambda x: remove_urls(x))

In [ ]:
def remove_html_tags(text):
    return re.sub(r'<.*?>', '', text)


clean_fake_df['text'] = clean_fake_df['text'].apply(lambda x: remove_html_tags(x))
clean_fake_df['title'] = clean_fake_df['title'].apply(lambda x: remove_html_tags(x))

clean_true_df['text'] = clean_true_df['text'].apply(lambda x: remove_html_tags(x))
clean_true_df['title'] = clean_true_df['title'].apply(lambda x: remove_html_tags(x))

In [ ]:
clean_true_df['text'] = clean_true_df['text'].apply(lambda x: unescape(x))
clean_true_df['title'] = clean_true_df['title'].apply(lambda x: unescape(x))

## Supprimer les chaînes de caractères vides


In [ ]:
clean_fake_df = clean_fake_df[clean_fake_df['text'].str.strip().astype(bool)]
clean_fake_df = clean_fake_df[clean_fake_df['title'].str.strip().astype(bool)]
clean_fake_df[clean_fake_df['title'].str.isspace()]

In [ ]:
clean_true_df = clean_true_df[clean_true_df['text'].str.strip().astype(bool)]
clean_true_df = clean_true_df[clean_true_df['title'].str.strip().astype(bool)]
clean_true_df[clean_true_df['title'].str.isspace()]

## Normaliser le format des dates


In [ ]:
clean_fake_df['date'] = pd.to_datetime(
    clean_fake_df['date'], errors='coerce', format='mixed'
)
clean_true_df['date'] = pd.to_datetime(
    clean_true_df['date'], errors='coerce', format='mixed'
)
clean_fake_df.sample(20)

In [ ]:
print(clean_fake_df['date'].isna().sum())
display(clean_fake_df[clean_fake_df['date'].isna()])

## Uniformiser la casse du texte


In [ ]:
clean_fake_df['text'] = clean_fake_df['text'].str.lower()
clean_fake_df['title'] = clean_fake_df['title'].str.lower()
clean_fake_df['subject'] = clean_fake_df['subject'].str.lower()

clean_true_df['text'] = clean_true_df['text'].str.lower()
clean_true_df['title'] = clean_true_df['title'].str.lower()
clean_true_df['subject'] = clean_true_df['subject'].str.lower()

clean_true_df.sample(20)

In [ ]:
fake_df['date'] = pd.to_datetime(fake_df['date'].str.strip('" '), errors='coerce', format='mixed')
print(fake_df['date'])



In [ ]:
fake_df.sample(50)